In [ ]:
from datasets import load_dataset, Dataset,DatasetDict
import pandas as pd

# 假設你的資料集是 CSV 格式
data = pd.read_csv('output/final_audio_paths.csv')

# 將資料轉換為 Hugging Face Dataset 格式
dataset = DatasetDict({
    "train": Dataset.from_pandas(data),
    "test": Dataset.from_pandas(data.sample(frac=0.2))  # 隨機取 20% 作為測試集
})


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, Audio
from sklearn.model_selection import train_test_split
from typing import Dict, Any

class AudioDatasetProcessor_V3:
    """
    處理音訊 CSV 的最終版本類別。
    - 讀取 'path' 與 'transcription' 欄位。
    - 將 'transcription' 重新命名為 'sentence'。
    - 產生與目標格式完全匹配的 DatasetDict。

    屬性:
        file_path (str): 輸入的 CSV 檔案路徑。
        target_sampling_rate (int): 目標音訊取樣率。
        test_size (float): 用於劃分測試集的資料比例。
        random_state (int): 隨機種子，確保可重複的資料切分。
    """

    def __init__(self, file_path: str, target_sampling_rate: int = 16000, test_size: float = 0.2, random_state: int = 42):
        """初始化 AudioDatasetProcessor_V3"""
        self.file_path = file_path
        self.target_sampling_rate = target_sampling_rate
        self.test_size = test_size
        self.random_state = random_state

    def _prepare_single_dataset(self, df: pd.DataFrame) -> Dataset:
        """
        將單一 DataFrame 轉換為具有指定取樣率 Audio 特徵的 Dataset。

        Args:
            df (pd.DataFrame): 包含 'audio' 和 'sentence' 欄位的 DataFrame。

        Returns:
            Dataset: 已處理的 Dataset 物件。
        """
        # 複製 'path' 欄位到 'audio'，此時 df 應包含 'path' 和 'sentence'
        temp_df = df.copy()
        temp_df['audio'] = temp_df['file']

        dataset = Dataset.from_pandas(temp_df)
        
        # 轉換 audio 欄位型別並移除多餘的 path 欄位
        dataset = dataset.cast_column("audio", Audio(sampling_rate=self.target_sampling_rate))


        return dataset

    def create_dataset_dict(self) -> DatasetDict:
        """
        執行完整的資料處理流程，包含欄位重新命名。

        Returns:
            DatasetDict: 包含 'train' 和 'test' splits 的最終物件。
        """
        # 讀取 CSV
        try:
            full_data = pd.read_csv(self.file_path)
        except FileNotFoundError:
            print(f"錯誤：找不到檔案 {self.file_path}")
            return None
            
        # 關鍵步驟 1: 指定來源需要的欄位是 'path' 和 'transcription'
        required_columns = ['transcription', 'file']
        if not all(col in full_data.columns for col in required_columns):
            raise ValueError(f"CSV 檔案缺少必要欄位。需要: {required_columns}")
        
        filtered_data = full_data[required_columns]

        # 使用處理後的 DataFrame 進行資料切分
        train_df, test_df = train_test_split(
             filtered_data,
            test_size=self.test_size,
            random_state=self.random_state
        )

        # 分別為訓練集和測試集準備 Dataset
        train_dataset = self._prepare_single_dataset(train_df)
        test_dataset = self._prepare_single_dataset(test_df)

        final_dataset_dict = DatasetDict({
            "train": train_dataset,
            "test": test_dataset
        })

        return final_dataset_dict

# --- 使用範例 ---
if __name__ == '__main__':
    # 假設您的 CSV 檔案名為 'final_audio_paths.csv'
    CSV_PATH = 'output/final_audio_paths.csv'
    
    # 實例化最終版本的處理器
    # 明確指定取樣率為 16000 Hz
    processor_v3 = AudioDatasetProcessor_V3(file_path=CSV_PATH, target_sampling_rate=16000)
    final_dataset = processor_v3.create_dataset_dict()

    if final_dataset:
        print("--- 最終版資料集結構 ---")
        print(final_dataset)
        
        print("\n--- Train Split Features ---")
        print(final_dataset['train'].features)

        print("\n--- 抽查第一筆訓練資料 (驗證最終格式) ---")
        print(final_dataset['train'][0])


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

from transformers import WhisperTokenizer
import pandas as pd
from datasets import Dataset, DatasetDict, Audio
from sklearn.model_selection import train_test_split
from typing import Dict, Any
from transformers import WhisperFeatureExtractor, WhisperTokenizer
class AudioDatasetProcessor_V3:
    """
    處理音訊 CSV 的最終版本類別。
    - 讀取 'path' 與 'transcription' 欄位。
    - 將 'transcription' 重新命名為 'sentence'。
    - 產生與目標格式完全匹配的 DatasetDict。

    屬性:
        file_path (str): 輸入的 CSV 檔案路徑。
        target_sampling_rate (int): 目標音訊取樣率。
        test_size (float): 用於劃分測試集的資料比例。
        random_state (int): 隨機種子，確保可重複的資料切分。
    """

    def __init__(self, file_path: str, target_sampling_rate: int = 16000, test_size: float = 0.2, random_state: int = 42):
        """初始化 AudioDatasetProcessor_V3"""
        self.file_path = file_path
        self.target_sampling_rate = target_sampling_rate
        self.test_size = test_size
        self.random_state = random_state

    def _prepare_single_dataset(self, df: pd.DataFrame) -> Dataset:
        """
        將單一 DataFrame 轉換為具有指定取樣率 Audio 特徵的 Dataset。

        Args:
            df (pd.DataFrame): 包含 'audio' 和 'sentence' 欄位的 DataFrame。

        Returns:
            Dataset: 已處理的 Dataset 物件。
        """
        # 複製 'path' 欄位到 'audio'，此時 df 應包含 'path' 和 'sentence'
        temp_df = df.copy()
        temp_df['audio'] = temp_df['file']

        dataset = Dataset.from_pandas(temp_df)
        
        # 轉換 audio 欄位型別並移除多餘的 path 欄位
        dataset = dataset.cast_column("audio", Audio(sampling_rate=self.target_sampling_rate))


        return dataset

    def create_dataset_dict(self) -> DatasetDict:
        """
        執行完整的資料處理流程，包含欄位重新命名。

        Returns:
            DatasetDict: 包含 'train' 和 'test' splits 的最終物件。
        """
        # 讀取 CSV
        try:
            full_data = pd.read_csv(self.file_path)
        except FileNotFoundError:
            print(f"錯誤：找不到檔案 {self.file_path}")
            return None
            
        # 關鍵步驟 1: 指定來源需要的欄位是 'path' 和 'transcription'
        required_columns = ['transcription', 'file']
        if not all(col in full_data.columns for col in required_columns):
            raise ValueError(f"CSV 檔案缺少必要欄位。需要: {required_columns}")
        
        filtered_data = full_data[required_columns]

        # 使用處理後的 DataFrame 進行資料切分
        train_df, test_df = train_test_split(
             filtered_data,
            test_size=self.test_size,
            random_state=self.random_state
        )

        # 分別為訓練集和測試集準備 Dataset
        train_dataset = self._prepare_single_dataset(train_df)
        test_dataset = self._prepare_single_dataset(test_df)

        final_dataset_dict = DatasetDict({
            "train": train_dataset,
            "test": test_dataset
        })

        return final_dataset_dict

# --- 使用範例 ---
if __name__ == '__main__':
    # 假設您的 CSV 檔案名為 'final_audio_paths.csv'
    CSV_PATH = 'output/final_audio_paths.csv'
    
    # 實例化最終版本的處理器
    # 明確指定取樣率為 16000 Hz
    processor_v3 = AudioDatasetProcessor_V3(file_path=CSV_PATH, target_sampling_rate=16000)
    final_dataset = processor_v3.create_dataset_dict()

    if final_dataset:
        print("--- 最終版資料集結構 ---")
        print(final_dataset)
        
        print("\n--- Train Split Features ---")
        print(final_dataset['train'].features)

        print("\n--- 抽查第一筆訓練資料 (驗證最終格式) ---")
        print(final_dataset['train'][0])

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="zh", task="transcribe")
from datasets import Audio

common_voice = final_dataset.cast_column("audio", Audio(sampling_rate=16000))
input_str = final_dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch
final_dataset = final_dataset.map(prepare_dataset, remove_columns=final_dataset.column_names["train"], num_proc=4)
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "hindi"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

# print(f"Input:                 {input_str}")
# print(f"Decoded w/ special:    {decoded_with_special}")
# print(f"Decoded w/out special: {decoded_str}")
# print(f"Are equal:             {input_str == decoded_str}")

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# ==============================================================================
# 檔案：train_final.py
# 描述：一個完整、高效、穩健的 Whisper 模型微調流程的最終版本。
# 核心策略：
# 1. 即時轉換 (.with_transform)：徹底解決記憶體不足與預處理過久的問題。
# 2. 背景預取 (dataloader_num_workers)：解決 CPU 與 I/O 瓶頸，最大化 GPU 使用率。
# 3. 全域定義 (Global Scope)：解決多核心處理時的 pickling 錯誤。
# 4. 智慧續練 (Smart Resuming)：自動從上次的檢查點恢復訓練。
# ==============================================================================

import torch
import pandas as pd
from typing import Any, Dict, List, Union
from dataclasses import dataclass
import evaluate
import os
from functools import partial

# --- Hugging Face 相關導入 ---
from huggingface_hub import login
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import Dataset, DatasetDict, Audio

# ==============================================================================
# 步驟 1: 將所有輔助類別與函式定義在「全域範圍」
# 這是為了確保在使用 dataloader_num_workers > 0 時，背景程序可以成功序列化 (pickle) 它們。
# ==============================================================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """處理語音到序列資料的 Data Collator，負責將樣本整理成批次並進行填充。"""
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

def prepare_dataset_batched(batch, feature_extractor, tokenizer):
    """將一批音訊和文本資料『即時』轉換為模型輸入格式。"""
    audio_list = batch["audio"]
    batch["input_features"] = feature_extractor(
        [x["array"] for x in audio_list], 
        sampling_rate=audio_list[0]["sampling_rate"]
    ).input_features
    batch["labels"] = tokenizer(batch["transcription"], max_length=448, truncation=True).input_ids
    return batch

def compute_metrics(pred, tokenizer):
    """在評估階段，計算並回傳 WER 指標。"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    metric = evaluate.load("wer")
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# ==============================================================================
# 步驟 2: 主執行流程
# ==============================================================================
def main():
    # --- 參數設定 ---
    CSV_PATH = 'output/final_audio_paths.csv'
    MODEL_NAME = "openai/whisper-small"
    LANGUAGE = "zh"
    TASK = "transcribe"
    OUTPUT_DIR = "./whisper-small-zh-finetune-final"

    # --- 載入 Processor 和模型 ---
    print("--- 步驟 1/4: 載入 Processor 和模型 ---")
    processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)
    model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    
    # --- 建立原始資料集 ---
    class AudioDatasetProcessor:
        def __init__(self, file_path: str, target_sampling_rate: int = 16000):
            self.file_path = file_path
            self.target_sampling_rate = target_sampling_rate
        def create_dataset(self) -> Dataset:
            full_data = pd.read_csv(self.file_path)
            dataset = Dataset.from_pandas(full_data)
            dataset = dataset.cast_column("file", Audio(sampling_rate=self.target_sampling_rate))
            dataset = dataset.rename_column("file", "audio")
            return dataset
            
    print("\n--- 步驟 2/4: 建立原始資料集並設定『即時轉換』---")
    audio_processor = AudioDatasetProcessor(file_path=CSV_PATH)
    full_dataset = audio_processor.create_dataset()
    common_voice = full_dataset.train_test_split(test_size=0.2, seed=42)
    
    # 使用 .with_transform() 確保記憶體穩定，訓練能立刻開始
    prepare_fn = partial(prepare_dataset_batched, feature_extractor=processor.feature_extractor, tokenizer=processor.tokenizer)
    vectorized_datasets = common_voice.with_transform(prepare_fn)
    print("即時轉換已設定。")

    # --- 建立訓練元件 ---
    print("\n--- 步驟 3/4: 建立訓練元件 (最終穩定運行版) ---")
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
    compute_metrics_fn = partial(compute_metrics, tokenizer=processor.tokenizer)

    # [最終修正]
    # 既然 dataloader_num_workers=0 是唯一能避免死鎖的方式，
    # 我們必須將訓練參數調整到與之匹配的保守水平，以避免 OOM 錯誤。
    training_args = Seq2SeqTrainingArguments(
        output_dir=OUTPUT_DIR,
        
        # 1. 大幅降低批次大小，這是避免 OOM 的核心
        per_device_train_batch_size=4,   # 從 32 或 16 大幅降至 4，這是一個極度安全的值
        per_device_eval_batch_size=4,    # 驗證批次也使用同樣的安全值
        
        # 2. 適度使用梯度累積，以穩定訓練
        # 有效批次大小為 4 * 4 = 16，這是一個不錯的平衡點
        gradient_accumulation_steps=4,
        
        # 3. 禁用多核心處理，這是確保程式不被掛起的關鍵
        dataloader_num_workers=0,
        
        # --- 其他參數維持不變 ---
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=5000,
        gradient_checkpointing=False,
        fp16=True,
        eval_strategy="steps",
        predict_with_generate=True,
        generation_max_length=225,
        save_steps=1000,
        eval_steps=1000,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=True,
        remove_unused_columns=False,
    )
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=vectorized_datasets["train"],
        eval_dataset=vectorized_datasets["test"],
        data_collator=data_collator,
        compute_metrics=compute_metrics_fn,
        tokenizer=processor.feature_extractor,
    )
    
    # --- 開始訓練 ---
    print("\n--- 步驟 4/4: 開始模型微調訓練 (使用最終穩定高效模式) ---")
    # 不帶參數的 .train() 會自動處理斷點續練，是最穩健的做法。
    trainer.train() 
    print("\n*** 訓練完成 ***")
    
    # --- 儲存最終模型 ---
    print("\n--- 正在儲存最終的最佳模型 ---")
    final_model_path = training_args.output_dir
    trainer.save_model(final_model_path)
    processor.save_pretrained(final_model_path)
    print(f"\n最終模型已儲存至：{final_model_path}")


if __name__ == '__main__':
    # 確保您已在終端機使用 `huggingface-cli login` 登入
    # 執行前建議重新啟動您的電腦，確保系統處於乾淨狀態
    main()

--- 步驟 1/4: 載入 Processor 和模型 ---

--- 步驟 2/4: 建立原始資料集並設定『即時轉換』---
即時轉換已設定。

--- 步驟 3/4: 建立訓練元件 (最終穩定運行版) ---


C:\Users\User\AppData\Local\Temp\ipykernel_9332\4225368149.py:153: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



--- 步驟 4/4: 開始模型微調訓練 (使用最終穩定高效模式) ---


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [2]:
import torch

def check_gpu_environment():
    """
    一個獨立的函式，用於檢查 PyTorch 的 GPU (CUDA) 環境是否設定正確。
    """
    print("--- PyTorch GPU 環境檢測報告 ---")
    
    is_cuda_available = torch.cuda.is_available()
    
    if is_cuda_available:
        print("\n[✅ 成功] `torch.cuda.is_available()` 回報：True")
        print("您的環境已準備好進行 GPU 加速訓練！")
        
        gpu_count = torch.cuda.device_count()
        print(f"\n[ℹ️ 資訊] 偵測到的可用 GPU 數量：{gpu_count} 張")
        
        current_device_id = torch.cuda.current_device()
        current_device_name = torch.cuda.get_device_name(current_device_id)
        print(f"[ℹ️ 資訊] 目前 PyTorch 將使用的 GPU：{current_device_name} (cuda:{current_device_id})")
        
    else:
        print("\n[❌ 失敗] `torch.cuda.is_available()` 回報：False")
        print("您的環境目前無法使用 GPU，訓練將會由 CPU 執行，速度會非常慢。")
        
    print("\n--- 檢測報告結束 ---")

if __name__ == '__main__':
    check_gpu_environment()

--- PyTorch GPU 環境檢測報告 ---

[✅ 成功] `torch.cuda.is_available()` 回報：True
您的環境已準備好進行 GPU 加速訓練！

[ℹ️ 資訊] 偵測到的可用 GPU 數量：1 張
[ℹ️ 資訊] 目前 PyTorch 將使用的 GPU：NVIDIA GeForce RTX 3060 Ti (cuda:0)

--- 檢測報告結束 ---


In [ ]:
import torch; print(torch.__version__)
import sys; print(sys.executable)

In [ ]:
import sys
import os

print("--- Python 系統路徑 (sys.path) 搜查報告 ---")
print("Python 會依照以下順序尋找函式庫：")
print("\n".join(sys.path))
print("-" * 50)

conflict_found = False
# 遍歷所有 Python 會尋找的路徑
for path in sys.path:
    # 我們只關心真實存在的資料夾
    if os.path.isdir(path):
        # 尋找任何可能造成衝突的檔案
        potential_conflict_file = os.path.join(path, "datasets.py")
        if os.path.exists(potential_conflict_file):
            print(f"[🚨 重大發現!] 在以下路徑中找到了名為 datasets.py 的衝突檔案：")
            print(f"==> {potential_conflict_file}")
            print("這就是造成您錯誤的根本原因！請立刻將此檔案重新命名或刪除。")
            conflict_found = True
            break # 找到一個就夠了

if not conflict_found:
    print("[✅ 正常] 在所有 Python 搜尋路徑中，未發現名為 datasets.py 的衝突檔案。")
    print("這表示問題可能出在函式庫安裝本身。請繼續執行下一步。")

print("\n--- 正在定位 `datasets` 函式庫的實際位置 ---")
try:
    import datasets
    # __file__ 屬性會告訴我們這個模組是從哪個檔案載入的
    print(f"[ℹ️ 資訊] 當您 `import datasets` 時，Python 實際載入的檔案是：")
    print(f"==> {datasets.__file__}")
    print("請檢查這個路徑是否在您的 .venv 虛擬環境的 site-packages 中。如果不是，代表您的環境設定有誤。")
except Exception as e:
    print(f"[❌ 錯誤] 甚至無法成功 `import datasets`，這強烈暗示您的安裝已損壞。")
    print(f"錯誤訊息: {e}")

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

In [ ]:
import torch

def check_gpu_environment():
    """
    一個獨立的函式，用於檢查 PyTorch 的 GPU (CUDA) 環境是否設定正確。
    """
    print("--- PyTorch GPU 環境檢測報告 ---")
    
    # 1. 核心檢測：torch.cuda.is_available()
    #    這是判斷 PyTorch 能否使用 GPU 的黃金標準。
    #    它會檢查 NVIDIA 驅動、CUDA 工具包是否都已安裝且被 PyTorch 正確偵測。
    is_cuda_available = torch.cuda.is_available()
    
    if is_cuda_available:
        print("\n[✅ 成功] `torch.cuda.is_available()` 回報：True")
        print("恭喜！您的 PyTorch 環境已成功偵測到並可以使用 GPU (CUDA)。")
        
        # 2. 獲取 GPU 數量
        gpu_count = torch.cuda.device_count()
        print(f"\n[ℹ️ 資訊] 偵測到的可用 GPU 數量：{gpu_count} 張")
        
        # 3. 獲取當前使用的 GPU 資訊
        #    PyTorch 預設使用第 0 張 GPU。
        current_device_id = torch.cuda.current_device()
        current_device_name = torch.cuda.get_device_name(current_device_id)
        print(f"[ℹ️ 資訊] 目前 PyTorch 預設使用的 GPU 編號：{current_device_id}")
        print(f"[ℹ️ 資訊] GPU 型號：{current_device_name}")
        
        # 4. 實戰驗證：嘗試將張量 (Tensor) 移至 GPU 並進行計算
        print("\n--- 正在進行實戰驗證 ---")
        try:
            # 創建一個張量並明確指定要使用的 CUDA 設備
            device = torch.device("cuda")
            tensor_cpu = torch.randn(3, 4)
            print(f"步驟 1: 在 CPU 上建立一個張量，位置：{tensor_cpu.device}")
            
            # 使用 .to(device) 將張量移至 GPU
            tensor_gpu = tensor_cpu.to(device)
            print(f"步驟 2: 已成功將張量複製到 GPU，新位置：{tensor_gpu.device}")
            
            # 在 GPU 上執行一個簡單的矩陣乘法
            result_gpu = torch.matmul(tensor_gpu.T, tensor_gpu)
            print("步驟 3: 已在 GPU 上成功完成矩陣乘法運算。")
            
            print("\n[✅ 結論] GPU 驗證成功！您的環境已準備好進行高效的深度學習訓練。")
            
        except Exception as e:
            print("\n[❌ 錯誤] 在實戰驗證過程中發生錯誤，這不應該發生。請檢查您的驅動程式。")
            print(f"錯誤訊息：{e}")
            
    else:
        print("\n[❌ 失敗] `torch.cuda.is_available()` 回報：False")
        print("很抱歉，您的 PyTorch 環境目前無法使用 GPU。")
        print("可能的原因包括：")
        print("  1. 您的電腦沒有安裝 NVIDIA 的 GPU。")
        print("  2. 您尚未安裝 NVIDIA 官方驅動程式。")
        print("  3. 您安裝的 PyTorch 版本是 CPU-only 版本。")
        print("  4. CUDA 工具包與 PyTorch 或驅動程式版本不相容。")
        print("\n建議操作：請確認以上幾點，並考慮在有 GPU 的環境（如 Google Colab）中執行您的訓練程式碼。")
        
    print("\n--- 檢測報告結束 ---")

# 執行檢測函式
if __name__ == '__main__':
    check_gpu_environment()

In [ ]:
import pandas as pd
import librosa
from datasets import Dataset, DatasetDict
from transformers import WhisperFeatureExtractor, WhisperTokenizer

# 假設你的資料集是 CSV 格式
data = pd.read_csv('output/final_audio_paths.csv')

# 將資料轉換為 Hugging Face Dataset 格式
dataset = DatasetDict({
    "train": Dataset.from_pandas(data),
    "test": Dataset.from_pandas(data.sample(frac=0.2))  # 隨機取 20% 作為測試集
})

# 定義 load_audio 函數
def load_audio(file_path):
    """加載音頻文件並返回音頻數組和取樣率"""
    audio_array, sampling_rate = librosa.load(file_path, sr=None)  # sr=None 保持原始取樣率
    return audio_array, sampling_rate

# 初始化 feature_extractor 和 tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="zh", task="transcribe")

# 定義數據集處理函數
def prepare_dataset(batch):
    audio = batch["file"]  # 假設這是音頻文件的路徑
    # 讀取音頻文件並轉換
    audio_array, sampling_rate = load_audio(audio)  # 使用自定義的 load_audio 函數
    batch["input_features"] = feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0]
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

# 使用 map 方法處理數據集
# dataset["train"] = dataset["train"].map(prepare_dataset, remove_columns=["transcription", "file"], num_proc=4)
dataset["train"] = dataset["train"].map(
    prepare_dataset,
    remove_columns=["transcription", "file"],
    batched=True,
    batch_size=16,  # 減少批次大小
    keep_in_memory=False,  # 設置為 False
    num_proc=4  # 使用 4 個進程
)

dataset["test"] = dataset["test"].map(
    prepare_dataset,
    remove_columns=["transcription", "file"],
    batched=True,
    batch_size=16,  # 減少批次大小
    keep_in_memory=False,  # 設置為 False
    num_proc=4  # 使用 4 個進程
)